# Create Data

In [2]:
import sys; sys.path.append("../../scripts/")
from generate_X import gen_trinmf_data
import matplotlib.pyplot as plt

kwkh=(5,3)
shape=(20,20)
data = gen_trinmf_data(shape=shape, 
                       kwkh=kwkh, 
                       factor_wh=(0.5, 1.0), 
                       factor_S=1,
                       random_state=10)
data.keys()

dict_keys(['X', 'W', 'S', 'H'])

In [3]:
X = data["X"]
Wtrue = data["W"]
Strue = data["S"]
Htrue = data["H"]
X.shape, Wtrue.shape, Strue.shape, Htrue.shape

((20, 20), (20, 5), (5, 3), (3, 20))

In [4]:
import numpy as np
Wcor= np.corrcoef(Wtrue.T)
maxcor = np.max(np.abs(Wcor[np.triu_indices(kwkh[0],k=2)]))
print(f'maximum correlation = {maxcor:.2f}')

maximum correlation = 0.39


# Initilize the Model

In [7]:
nmfk_params = {
    "n_perturbs":64,
    "n_iters":2000,
    "epsilon":0.01,
    "n_jobs":-1,
    "init":"nnsvd",
    "use_gpu":False,
    "save_path":"../../results/",
    "verbose":True,
    "sill_thresh":0.8,
    "nmf_method":"nmf_fro_mu", 
    "perturb_type":"uniform", 
    "calculate_error":True,
    "pruned":True,
    "predict_k":True,
    "predict_k_method":"sill",
    "transpose":False,
    "mask":None,
    "use_consensus_stopping":False,
    "calculate_pac":True,
    "consensus_mat":True,
    "simple_plot":True,
    "collect_output":True
}

tri_nmfk_params = {
    "experiment_name":"TriNMFk",
    "nmfk_params":nmfk_params,
    "nmf_verbose":False,
    "use_gpu":True,
    "n_jobs":-1,
    "mask":None,
    "use_consensus_stopping":False,
    "alpha":(0,0),
    "n_iters":100,
    "n_inits":10,
    "joblib_backend":"multiprocessing",
    "pruned":False,
    "transpose":False,
    "verbose":True
}

In [8]:
from TELF.factorization import TriNMFk
model = TriNMFk(**tri_nmfk_params)

Performing NMF with  nmf_fro_mu


/Users/maksim/opt/miniconda3/envs/TELF/lib/python3.8/site-packages/TELF-0.0.1-py3.8.egg/TELF/factorization/utilities/organize_n_jobs.py:10: UserWarning: No GPU found! Using CPUs
/Users/maksim/opt/miniconda3/envs/TELF/lib/python3.8/site-packages/TELF-0.0.1-py3.8.egg/TELF/factorization/NMFk.py:392: UserWarning: calculate_error is True! Error calculation can make the runtime longer and take up more memory space!


# Perform NMFk First

In [9]:
Ks = range(1,8,1)
note = "This the the NMFk portion of the TriNMFk method!"
results = model.fit_nmfk(X, Ks, note)

[Parallel(n_jobs=7)]: Using backend MultiprocessingBackend with 7 concurrent workers.
/Users/maksim/Desktop/Code/telf-public/.eggs/numpy-1.22.0-py3.8-macosx-10.9-x86_64.egg/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/maksim/Desktop/Code/telf-public/.eggs/numpy-1.22.0-py3.8-macosx-10.9-x86_64.egg/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/maksim/Desktop/Code/telf-public/.eggs/numpy-1.22.0-py3.8-macosx-10.9-x86_64.egg/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/maksim/opt/miniconda3/envs/TELF/lib/python3.8/site-packages/TELF-0.0.1-py3.8.egg/TELF/factorization/NMFk.py:279: RuntimeWarning: [tELF]: Encountered unknown logging metric "pac"
[Parallel(n_jobs=7)]: Done   2 out of   7 | elapsed:   10.1s remaining:   25.2s
/Users/maksim/Desktop/Code/tel

In [10]:
results.keys()

dict_keys(['time', 'k_predict'])

# Here now look at the outputs and choose k1 and k2 manually

In [11]:
! ls ../../results/TriNMFk_64perts_2000iters_0.01eps_nnsvd-init

FINAL_k=1-7.png            k_1_7_cophenetic_coeff.png
WH_k=1.npz                 k_1_con_mat.png
WH_k=2.npz                 k_2_con_mat.png
WH_k=3.npz                 k_3_con_mat.png
WH_k=4.npz                 k_4_con_mat.png
WH_k=5.npz                 k_5_con_mat.png
WH_k=6.npz                 k_6_con_mat.png
WH_k=7.npz                 k_7_con_mat.png
experiment.log


# Perform TriNMFk with the selected k1 and k2

In [12]:
k1k2=(5,3)
tri_nmfk_results = model.fit_tri_nmfk(X, k1k2)

[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    7.7s remaining:    5.1s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    9.7s finished


In [13]:
tri_nmfk_results.keys()

dict_keys(['W', 'S', 'H', 'errors'])

In [14]:
W = tri_nmfk_results["W"]
S = tri_nmfk_results["S"]
H = tri_nmfk_results["H"]

In [15]:
W.shape, S.shape, H.shape

((20, 5), (5, 3), (3, 20))